In [1]:
# Basic import
import os
import sys
import json
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cv2

In [2]:
# Helper function
def writeProgress(msg, count, total):
    sys.stdout.write(msg + "{:.2%}\r".format(count/total))
    sys.stdout.flush()
    
def newPath(path):
    if not os.path.isdir(path):
        os.mkdir(path)

def read_json(src_path):
    with open(src_path, 'r') as json_file:
        data = json.load(json_file)
    return data

def write_json(data,dst_path):
    with open(dst_path, 'w') as outfile:
        json.dump(data, outfile)

## 2019 movies

In [5]:
PATH = './SplitTrailers_crop/'
trailers = os.listdir(PATH)
trailers.sort()
print(trailers[:10])
# trailers = trailers[1:]
print(len(trailers), trailers[:10])

['21bridgesmovie', '47metersdown', 'abeautifuldaymovie', 'abominablemovie', 'adastramovie', 'adogsjourneymovie', 'aftermathmovie', 'aftermovie', 'ahiddenlifefilm', 'alitamovie']
165 ['21bridgesmovie', '47metersdown', 'abeautifuldaymovie', 'abominablemovie', 'adastramovie', 'adogsjourneymovie', 'aftermathmovie', 'aftermovie', 'ahiddenlifefilm', 'alitamovie']


## 2018 movies

In [16]:
PATH = './2018SplitTrailers_crop/'
df_onehot = pd.read_csv('./csv/filename2genreMat_2018.csv')
df_onehot

,id,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,...,History,Horror,Music,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,tt0328810,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,tt10005184,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,tt10017502,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,tt10043732,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,tt10048096,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1400,tt9866700,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
1401,tt9879080,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
1402,tt9891764,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1403,tt9904014,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [17]:
trailers = df_onehot['id'].tolist()
print(len(trailers), trailers[:10])

1405 ['tt0328810', 'tt10005184', 'tt10017502', 'tt10043732', 'tt10048096', 'tt10059624', 'tt10077620', 'tt10116528', 'tt10160782', 'tt10178206']


# A_AverageShotLength

In [6]:
A_li = []
for movie in trailers:
    frameCount = []
    print('Running:', movie)
    
    for scene in os.listdir(PATH + movie):
        framePath = PATH + movie + '/' +scene + '/frames/'
        frameCount.append(len(os.listdir(framePath)))
    
#     print(frameCount)
    avg = np.mean(frameCount)
    std = np.std(frameCount)
    print(avg, std)
    
    A_li.append([avg, std])

Running: 21bridgesmovie
33.73873873873874 30.367927519550022
Running: 47metersdown
34.80487804878049 23.842183882819985
Running: abeautifuldaymovie
42.24096385542169 41.67649083389351
Running: abominablemovie
32.69911504424779 26.41864618926357
Running: adastramovie
59.215384615384615 55.803637131998514
Running: adogsjourneymovie
34.63247863247863 29.109753495301806
Running: aftermathmovie
35.69230769230769 26.611910622543594
Running: aftermovie
58.276595744680854 54.90405640467415
Running: ahiddenlifefilm
53.49152542372882 54.73615385054971
Running: alitamovie
35.78431372549019 29.192272497163273
Running: angelhasfallen
36.42156862745098 27.8171326366979
Running: angrybirdsmovie
34.65 30.91052948969547
Running: annabellemovie
52.36363636363637 45.788259830354015
Running: annamovie
37.91011235955056 33.61267833617541
Running: apollo11movie
65.6829268292683 58.86505403510322
Running: arcticdogsmovie
37.0 32.503393487992824
Running: arcticmovie
72.625 94.80177863385717
Running: artofraci

In [7]:
A_arr = np.asarray(A_li)
np.save('./npy/A_arr.npy', A_arr)
len(A_arr)

165

# B_ColorVariance
* https://docs.opencv.org/3.4/de/d25/imgproc_color_conversions.html#color_convert_rgb_luv
* https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.det.html

In [8]:
B_li = []
for movie in trailers: #order:
    rho = []
    print('Running:', movie)
    
    for scene in os.listdir(PATH + movie):
        framePath = PATH + movie + '/' + scene + '/frames/'
        frameList = [int(name.split('.')[0]) for name in os.listdir(framePath) if not name.startswith('.')]
        frameList.sort()
        keyframe = framePath + '/' + str(frameList[len(frameList)//2]) + '.jpg'
        img = cv2.imread(keyframe)
        luv = cv2.cvtColor(img, cv2.COLOR_BGR2Luv)
        l = luv[:,:,0].ravel()
        u = luv[:,:,1].ravel()
        v = luv[:,:,2].ravel()
        x = np.asarray((l, u, v))
        cov = np.cov(x)
        det = np.linalg.det(cov)
        rho.append(det)
    
    avg = np.mean(rho)
    std = np.std(rho)
    print(avg, std)
    
    B_li.append([avg, std])

Running: 21bridgesmovie
1096283.5315682662 5072586.866352188
Running: 47metersdown
11937779.160579428 30622012.29551432
Running: abeautifuldaymovie
2680926.332467057 7795519.214254663
Running: abominablemovie
39152077.72039263 81068156.03628635
Running: adastramovie
956515.9899208385 2935082.754121198
Running: adogsjourneymovie
2688809.223522822 4606428.453921334
Running: aftermathmovie
737843.0010296163 1447867.703815384
Running: aftermovie
5730162.1264876025 14299287.334890012
Running: ahiddenlifefilm
2921107.3612849615 6265612.025954831
Running: alitamovie
2531454.7904955284 6295080.15899654
Running: angelhasfallen
1209756.4450619791 4667335.491197369
Running: angrybirdsmovie
369915511.07856596 751228830.3413738
Running: annabellemovie
4579331.774892684 34055753.876763575
Running: annamovie
9961093.345247298 30478166.89014458
Running: apollo11movie
6274530.625040619 15502910.006607102
Running: arcticdogsmovie
224237674.49668354 949842786.3804522
Running: arcticmovie
419908.793015233

In [9]:
B_arr = np.asarray(B_li)
np.save('./npy/B_arr.npy', B_arr)
len(B_arr)

165

# C_MotionContent
* https://blog.gtwang.org/programming/opencv-motion-detection-and-tracking-tutorial/

In [10]:
C_li = []
for movie in trailers: # [900:]:
    moving_li = []
    print('Running:', movie)
    
    for scene in os.listdir(PATH + movie):
    
        video  = PATH + movie + '/' + scene + '/clip.avi'

        cap = cv2.VideoCapture(video)

        w = int(cap.get(3))
        h = int(cap.get(4))

        # 計算畫面面積
        area = w * h

        # 初始化平均影像
        ret, frame = cap.read()

        avg = cv2.blur(frame,(4, 4))
        avg_float = np.float32(avg)
#         plt.imshow(frame)
#         plt.show()

        while(cap.isOpened()):
            # 讀取一幅影格
            ret, frame = cap.read()

            # 若讀取至影片結尾，則跳出
            if ret == False:
                break

            # 模糊處理
            blur = cv2.blur(frame, (4, 4))

            # 計算目前影格與平均影像的差異值
            diff = cv2.absdiff(frame, avg)

            # 將圖片轉為灰階
            gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)

            # 篩選出變動程度大於門檻值的區域
            ret, thresh = cv2.threshold(gray, 25, 255, cv2.THRESH_BINARY)

            # 使用型態轉換函數去除雜訊
            kernel = np.ones((5, 5), np.uint8)
            thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)
            thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)

            # 產生等高線
            cnts, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            moving = 0
            for c in cnts:
                moving += cv2.contourArea(c)
                
            moving_li.append(moving/area)
            
#             for c in cnts:
#                 # 忽略太小的區域
#                 if cv2.contourArea(c) < 1000:
#                     continue


#             # 畫出等高線（除錯用）
#             cv2.drawContours(frame, cnts, -1, (0, 255, 255), 2)
#             plt.imshow(frame)
#             plt.show()

            # 更新平均影像
            cv2.accumulateWeighted(blur, avg_float, 0.01)
            avg = cv2.convertScaleAbs(avg_float)

        cap.release()
        cv2.destroyAllWindows()
    
    avg = np.mean(moving_li)
    std = np.std(moving_li)
    print(avg, std)
    
    C_li.append([avg, std])

Running: 21bridgesmovie
0.18675777789485098 0.24562201559469524
Running: 47metersdown
0.24560857604689956 0.24974261700357553
Running: abeautifuldaymovie
0.23793018889242104 0.3072599992258611
Running: abominablemovie
0.2518363644696767 0.2845149696465054
Running: adastramovie
0.25311789569335913 0.3009523702819392
Running: adogsjourneymovie
0.33599442968175286 0.349168693945698
Running: aftermathmovie
0.20429768996170855 0.2599513991686895
Running: aftermovie
0.212973970075358 0.2316550068940771
Running: ahiddenlifefilm
0.25630518498921984 0.23970192238709184
Running: alitamovie
0.2673263193152564 0.26963690207016533
Running: angelhasfallen
0.25070263657957015 0.28292585750320864
Running: angrybirdsmovie
0.3415629566501276 0.34148056657907155
Running: annabellemovie
0.09246866145816199 0.13234750118080257
Running: annamovie
0.3236956404202224 0.29021756294467305
Running: apollo11movie
0.18749989547658708 0.23665064533322383
Running: arcticdogsmovie
0.31903532445453675 0.33199816308151

In [11]:
C_arr = np.asarray(C_li)
np.save('./npy/C_arr.npy', C_arr)
len(C_arr)

165

# D_LightingKey

In [12]:
D_li = []
for movie in trailers:
    lk = []
    print('Running:', movie)
    
    for scene in os.listdir(PATH + movie):
        framePath = PATH + movie + '/' + scene + '/frames/'
        frameList = [int(name.split('.')[0]) for name in os.listdir(framePath) if not name.startswith('.')]
        frameList.sort()
        keyframe = framePath + '/' + str(frameList[len(frameList)//2]) + '.jpg'
        img = cv2.imread(keyframe)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        value = hsv[:,:,2].ravel()
        mean = np.mean(value)
        std = np.std(value)
        lk.append(mean*std)

    avg = np.mean(lk)
    std = np.std(lk)
    print(avg, std)
    
    D_li.append([avg, std])

Running: 21bridgesmovie
1437.890474609901 1191.7232366195374
Running: 47metersdown
3832.149222456684 3285.912168339273
Running: abeautifuldaymovie
3787.7294532382807 2395.3735756891256
Running: abominablemovie
4908.7643296277565 2895.099930822099
Running: adastramovie
3213.4972316700614 2403.1571906203135
Running: adogsjourneymovie
3225.348524584389 2086.338315789782
Running: aftermathmovie
3212.997036429973 2613.398541739849
Running: aftermovie
3180.640515062546 2106.905406681398
Running: ahiddenlifefilm
5018.189068005628 2986.0754864720816
Running: alitamovie
2845.366840575144 2350.3135919548176
Running: angelhasfallen
3686.115581034615 2869.0750183839014
Running: angrybirdsmovie
7726.730432469832 2614.3218269550307
Running: annabellemovie
1387.4944020693085 1821.0507450331909
Running: annamovie
4568.54899177322 2762.151807233616
Running: apollo11movie
4119.307187002707 2746.299971683731
Running: arcticdogsmovie
7218.85843914613 3487.3198107006574
Running: arcticmovie
2704.2718219421

In [13]:
D_arr = np.asarray(D_li)
np.save('./npy/D_arr.npy', D_arr)
len(D_arr)

165

# Results

In [14]:
# reload computed values
A_arr = np.load('./npy/A_arr.npy')
B_arr = np.load('./npy/B_arr.npy')
C_arr = np.load('./npy/C_arr.npy')
D_arr = np.load('./npy/D_arr.npy')

In [15]:
print(A_arr.shape, B_arr.shape, C_arr.shape, D_arr.shape)

(165, 2) (165, 2) (165, 2) (165, 2)


In [17]:
# First time succeed!
results = pd.DataFrame({
    'Movie': trailers,
    'A_avg': A_arr[:,0],
    'A_std': A_arr[:,1],
    'B_avg': B_arr[:,0],
    'B_std': B_arr[:,1],
    'C_avg': C_arr[:,0],
    'C_std': C_arr[:,1],
    'D_avg': D_arr[:,0],
    'D_std': D_arr[:,1]
})
results

,Movie,A_avg,A_std,B_avg,B_std,C_avg,C_std,D_avg,D_std
0,21bridgesmovie,33.738739,30.367928,1.096284e+06,5.072587e+06,0.186758,0.245622,1437.890475,1191.723237
1,47metersdown,34.804878,23.842184,1.193778e+07,3.062201e+07,0.245609,0.249743,3832.149222,3285.912168
2,abeautifuldaymovie,42.240964,41.676491,2.680926e+06,7.795519e+06,0.237930,0.307260,3787.729453,2395.373576
3,abominablemovie,32.699115,26.418646,3.915208e+07,8.106816e+07,0.251836,0.284515,4908.764330,2895.099931
4,adastramovie,59.215385,55.803637,9.565160e+05,2.935083e+06,0.253118,0.300952,3213.497232,2403.157191
...,...,...,...,...,...,...,...,...,...
160,wrinklestheclown,39.423529,26.872412,1.253078e+07,3.489942e+07,0.218662,0.281909,3848.612858,3919.601640
161,xmenmovies,41.549296,33.954729,4.791049e+06,1.487502e+07,0.319549,0.279627,2953.689532,2301.564008
162,yardiefilm,40.670330,40.923959,8.502874e+05,2.444444e+06,0.176796,0.197451,2192.990325,1674.572914
163,yesterdaymovie,31.888112,23.459265,1.347190e+07,2.971155e+07,0.180151,0.253020,4528.466900,3376.180082


In [18]:
results.to_csv('./csv/computed_2019.csv', index = 0)

In [19]:
# Log
results = pd.DataFrame({
    'Movie': trailers,
    'A_avg': np.log(A_arr[:,0]),
    'A_std': np.log(A_arr[:,1]),
    'B_avg': np.log(B_arr[:,0]),
    'B_std': np.log(B_arr[:,1]),
    'C_avg': C_arr[:,0],
    'C_std': C_arr[:,1],
    'D_avg': np.log(D_arr[:,0]),
    'D_std': np.log(D_arr[:,1])
})
results

,Movie,A_avg,A_std,B_avg,B_std,C_avg,C_std,D_avg,D_std
0,21bridgesmovie,3.518647,3.413387,13.907436,15.439361,0.186758,0.245622,7.270932,7.083156
1,47metersdown,3.549758,3.171456,16.295219,17.237230,0.245609,0.249743,8.251181,8.097400
2,abeautifuldaymovie,3.743390,3.729937,14.801673,15.869060,0.237930,0.307260,8.239522,7.781294
3,abominablemovie,3.487348,3.274070,17.482964,18.210801,0.251836,0.284515,8.498778,7.970775
4,adastramovie,4.081181,4.021839,13.771053,14.892246,0.253118,0.300952,8.075115,7.784539
...,...,...,...,...,...,...,...,...,...
160,wrinklestheclown,3.674363,3.291100,16.343698,17.367981,0.218662,0.281909,8.255468,8.273745
161,xmenmovies,3.726881,3.525028,15.382260,16.515194,0.319549,0.279627,7.990810,7.741344
162,yardiefilm,3.705499,3.711716,13.653330,14.709328,0.176796,0.197451,7.693021,7.423313
163,yesterdaymovie,3.462233,3.155266,16.416116,17.207047,0.180151,0.253020,8.418139,8.124500


In [20]:
results.to_csv('./csv/computed_2019_log.csv', index = 0)

# Remove invalid movies

In [22]:
with open('./Check2018_id.txt', 'r') as f:
    lines = [line.rstrip() for line in f]

In [23]:
lines

['tt9446774',
 'tt4883336',
 'tt8152842',
 'tt2709692',
 'tt2396557',
 'tt9417976',
 'tt9183176',
 'tt6774588',
 'tt8973954',
 'tt8824506',
 'tt4660378',
 'tt8997108',
 'tt8253812',
 'tt7896034',
 'tt6693892',
 'tt9104922',
 'tt2396557',
 'tt8906732',
 'tt9417976',
 'tt8493136',
 'tt9183176',
 'tt6774588',
 'tt2051958',
 'tt8973954',
 'tt6098808',
 'tt9318706',
 'tt4660378',
 'tt9635722',
 'tt9289116',
 'tt7466770',
 'tt8336974',
 'tt8819182',
 'tt8564100',
 'tt7581080',
 'tt4581774',
 'tt9037262',
 'tt8137788',
 'tt9386390',
 'tt6151592',
 'tt4173184',
 'tt1828172',
 'tt8887736',
 'tt7842870',
 'tt8493136',
 'tt6774588',
 'tt9417976',
 'tt5824110',
 'tt9627094',
 'tt9455514',
 'tt6433034',
 'tt8947034',
 'tt8288798',
 'tt9318706',
 'tt9747894',
 'tt9475908',
 'tt6181262',
 'tt9367778',
 'tt8922582',
 'tt9604418',
 'tt8368346',
 'tt9138208',
 'tt6098808',
 'tt8386692',
 'tt7165654',
 'tt7999860',
 'tt8364460',
 'tt7533756',
 'tt6580062',
 'tt8973954',
 'tt9183176',
 'tt5775536',
 'tt71

In [24]:
filtered = results[~results.Movie.isin(lines)]
filtered

,Movie,A_avg,A_std,B_avg,B_std,C_avg,C_std,D_avg,D_std
0,tt0328810,4.308438,4.351987,15.820982,15.725770,0.336424,0.335943,9.025204,8.446634
1,tt10005184,4.852898,5.128792,13.870248,14.762869,0.099732,0.170803,8.552695,7.614880
2,tt10017502,3.464816,2.680851,14.985025,15.618921,0.268349,0.325218,8.206244,7.652673
3,tt10043732,3.729799,3.344649,15.602536,16.654683,0.315923,0.330436,8.339235,8.113956
4,tt10048096,3.349202,3.322474,16.273237,16.918459,0.296425,0.314294,8.349771,7.793370
...,...,...,...,...,...,...,...,...,...
1400,tt9866700,4.941889,4.402468,13.849033,14.518005,0.115961,0.170274,8.603843,8.232477
1401,tt9879080,4.426478,3.947812,14.387161,14.694599,0.058649,0.080815,7.524463,7.256423
1402,tt9891764,3.487894,2.994688,15.839656,16.387069,0.243142,0.287351,8.387383,7.987073
1403,tt9904014,4.687189,4.298575,12.841054,13.330432,0.339983,0.243896,7.993096,7.750461


In [25]:
filtered.to_csv('./csv/computed_2018_filtered_log.csv', index = 0)

## Normalize
* https://scikit-learn.org/stable/modules/preprocessing.html

In [21]:
from sklearn import preprocessing

### MinMaxScaler

In [22]:
def normalizing(data):
    scaler = preprocessing.MinMaxScaler()
    scaler.fit(data)
    print('Data range:', scaler.data_range_)
    return scaler.transform(data)

In [23]:
results = pd.read_csv('./csv/computed_2019_log.csv')
results

,Movie,A_avg,A_std,B_avg,B_std,C_avg,C_std,D_avg,D_std
0,21bridgesmovie,3.518647,3.413387,13.907436,15.439361,0.186758,0.245622,7.270932,7.083156
1,47metersdown,3.549758,3.171456,16.295219,17.237230,0.245609,0.249743,8.251181,8.097400
2,abeautifuldaymovie,3.743390,3.729937,14.801673,15.869060,0.237930,0.307260,8.239522,7.781294
3,abominablemovie,3.487348,3.274070,17.482964,18.210801,0.251836,0.284515,8.498778,7.970775
4,adastramovie,4.081181,4.021839,13.771053,14.892246,0.253118,0.300952,8.075115,7.784539
...,...,...,...,...,...,...,...,...,...
160,wrinklestheclown,3.674363,3.291100,16.343698,17.367981,0.218662,0.281909,8.255468,8.273745
161,xmenmovies,3.726881,3.525028,15.382260,16.515194,0.319549,0.279627,7.990810,7.741344
162,yardiefilm,3.705499,3.711716,13.653330,14.709328,0.176796,0.197451,7.693021,7.423313
163,yesterdaymovie,3.462233,3.155266,16.416116,17.207047,0.180151,0.253020,8.418139,8.124500


In [24]:
cols = results.columns[1:].tolist()
cols

['A_avg', 'A_std', 'B_avg', 'B_std', 'C_avg', 'C_std', 'D_avg', 'D_std']

In [25]:
for col in cols:
    arr = np.asarray(results[col].tolist()).reshape(-1, 1)
    results[col+'_minmaxnorm'] = normalizing(arr)

Data range: [3.4382737]
Data range: [4.16164127]
Data range: [10.13247484]
Data range: [10.7289231]
Data range: [0.35217662]
Data range: [0.24881802]
Data range: [1.75468836]
Data range: [1.20647095]


In [26]:
results.to_csv('./csv/computed_minmaxnorm_2019_log.csv', index = 0)
results

,Movie,A_avg,A_std,B_avg,B_std,C_avg,C_std,D_avg,D_std,A_avg_minmaxnorm,A_std_minmaxnorm,B_avg_minmaxnorm,B_std_minmaxnorm,C_avg_minmaxnorm,C_std_minmaxnorm,D_avg_minmaxnorm,D_std_minmaxnorm
0,21bridgesmovie,3.518647,3.413387,13.907436,15.439361,0.186758,0.245622,7.270932,7.083156,0.064317,0.212065,0.425476,0.512305,0.303963,0.455250,0.029611,0.013163
1,47metersdown,3.549758,3.171456,16.295219,17.237230,0.245609,0.249743,8.251181,8.097400,0.073366,0.153932,0.661132,0.679877,0.471069,0.471811,0.588256,0.853833
2,abeautifuldaymovie,3.743390,3.729937,14.801673,15.869060,0.237930,0.307260,8.239522,7.781294,0.129683,0.288129,0.513731,0.552355,0.449266,0.702974,0.581612,0.591825
3,abominablemovie,3.487348,3.274070,17.482964,18.210801,0.251836,0.284515,8.498778,7.970775,0.055214,0.178589,0.778354,0.770619,0.488753,0.611561,0.729362,0.748879
4,adastramovie,4.081181,4.021839,13.771053,14.892246,0.253118,0.300952,8.075115,7.784539,0.227927,0.358270,0.412016,0.461310,0.492392,0.677623,0.487916,0.594514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,wrinklestheclown,3.674363,3.291100,16.343698,17.367981,0.218662,0.281909,8.255468,8.273745,0.109606,0.182681,0.665917,0.692064,0.394556,0.601089,0.590700,1.000000
161,xmenmovies,3.726881,3.525028,15.382260,16.515194,0.319549,0.279627,7.990810,7.741344,0.124881,0.238892,0.571030,0.612579,0.681022,0.591916,0.439871,0.558712
162,yardiefilm,3.705499,3.711716,13.653330,14.709328,0.176796,0.197451,7.693021,7.423313,0.118662,0.283751,0.400398,0.444261,0.275676,0.261650,0.270160,0.295108
163,yesterdaymovie,3.462233,3.155266,16.416116,17.207047,0.180151,0.253020,8.418139,8.124500,0.047910,0.150041,0.673064,0.677064,0.285204,0.484984,0.683406,0.876296


### StandardScaler

In [ ]:
def normalizing(data):
    scaler = preprocessing.StandardScaler()
    scaler.fit(data)
    print('Mean:', scaler.mean_)
    print('Scale:', scaler.scale_)
    return scaler.transform(data)

In [ ]:
results = pd.read_csv('./csv/computed_2018.csv')
results

In [ ]:
cols = results.columns[1:].tolist()
cols

In [ ]:
for col in cols:
    arr = np.asarray(results[col].tolist()).reshape(-1, 1)
    results[col+'_stdnorm'] = normalizing(arr)

In [ ]:
results.to_csv('./csv/computed_stdnorm_2018.csv', index = 0)
results

### MaxAbsScaler

In [ ]:
def normalizing(data):
    scaler = preprocessing.MaxAbsScaler()
    scaler.fit(data)
    print('Scale:', scaler.scale_)
    return scaler.transform(data)

In [ ]:
results = pd.read_csv('./csv/computed_2018.csv')
results

In [ ]:
cols = results.columns[1:].tolist()
cols

In [ ]:
for col in cols:
    arr = np.asarray(results[col].tolist()).reshape(-1, 1)
    results[col+'_maxabsnorm'] = normalizing(arr)

In [ ]:
results.to_csv('./csv/computed_maxabsnorm_2018.csv', index = 0)
results

### RobustScaler

In [ ]:
def normalizing(data):
    scaler = preprocessing.RobustScaler()
    scaler.fit(data)
    print('Scale:', scaler.scale_)
    return scaler.transform(data)

In [ ]:
results = pd.read_csv('./csv/computed_2018.csv')
results

In [ ]:
cols = results.columns[1:].tolist()
cols

In [ ]:
for col in cols:
    arr = np.asarray(results[col].tolist()).reshape(-1, 1)
    results[col+'_robnorm'] = normalizing(arr)

In [ ]:
results.to_csv('./csv/computed_robnorm_2018.csv', index = 0)
results